<a href="https://colab.research.google.com/github/https-Aelius/AI_prac/blob/main/Forecast_AI_training_prac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
pip install scikit-learn pandas_datareader

In [2]:
import pandas #https://colab.research.google.com/drive/1hftHUQd1bL7sdW-iNGalrGyBwFbq5yN7datareader.data as pdr
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
import yfinance as yf
import pytz
from datetime import datetime as dt

# Data Download

In [4]:
tz = pytz.timezone("America/New_York")
start_date = tz.localize(dt(2022,1,1))
end_date = tz.localize(dt(2022,6,14))


df = yf.download(['AAPL'], start=start_date, end=end_date, auto_adjust=True)

#df.to_csv("stocks.csv")




ERROR:yfinance:Failed to get ticker 'AAPL' reason: unexpected character: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed


# Dataset creation

In [5]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2022-01-03,178.645645,179.499574,174.425140,174.542917,104487900
2022-01-04,176.378357,179.558473,175.809076,179.254206,99310400
2022-01-05,171.686707,176.839663,171.411883,176.290017,94537600
2022-01-06,168.820679,172.059683,168.467333,169.507737,96904000
2022-01-07,168.987518,170.921105,167.868591,169.694211,86709100


In [6]:
df["Diff"] = df.Close.diff()
df["SMA_2"] = df.Close.rolling(2).mean()

df["Force_Index"] = df["Close"] * df["Volume"]

df["y"] = df["Diff"].apply(lambda x: 1 if x > 0 else 0).shift(-1)

df = df.drop(
   ["Open", "High", "Low", "Close", "Volume", "Diff"],
   axis=1,
).dropna()

#df.head()
# # # # print(df)
X = df.drop(["y"], axis=1).values
y = df["y"].values
X_train, X_test, y_train, y_test = train_test_split(
   X,
   y,
   test_size=0.2,
   shuffle=False,
)

In [7]:
df.head()

Price,SMA_2,Force_Index,y
Ticker,,,
Date,,,
2022-01-04,177.512001,1.751621e+10,0.0
2022-01-05,174.032532,1.623085e+10,0.0
2022-01-06,170.253693,1.635940e+10,1.0
2022-01-07,168.904099,1.465276e+10,1.0
2022-01-10,168.997330,1.804415e+10,1.0


In [8]:
X_train.shape

(88, 2)

In [9]:
input_shape=(X_train.shape[1], 1)
print(input_shape)

(2, 1)


In [10]:
y

array([0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 0., 0., 0.])

# Training

In [11]:
clf = make_pipeline(StandardScaler(), MLPClassifier(random_state=0, shuffle=False))
clf.fit(
   X_train,
   y_train,
)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.45454545454545453


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.])

In [13]:
y_test

array([0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 0.])